In [2]:
import pandas as pd 

In [4]:
df = pd.read_csv("./data/Kickstarter_merged.csv")

In [9]:
# convert 'video' to a binary categorical variable
df['video'].value_counts()
df['has_video'] = df['video'].apply(lambda x: 0 if pd.isnull(x) else 1)
df['has_video'].value_counts()

1    1404
0     728
Name: has_video, dtype: int64